## 先安装依赖：

pip install alibabacloud_devops20210625

In [ ]:
# pip install --index-url=https://mirrors.aliyun.com/pypi/simple/ --trusted-host=mirrors.aliyun.com --default-timeout=30 alibabacloud_devops20210625

## 配置ALI云账户信息

In [ ]:
import os

from typing import List

from alibabacloud_devops20210625.client import Client as devops20210625Client
from alibabacloud_tea_openapi import models as open_api_models
from alibabacloud_tea_util import models as util_models
from alibabacloud_tea_util.client import Client as UtilClient
from alibabacloud_devops20210625 import models as devops_20210625_models
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logging.info("萨大王的")

# Mac用户可以通过zsh 或者 bash来设置全局环境变量：
# 比如：
# echo "export ALIBABA_CLOUD_ACCESS_KEY_ID=ABCD1234" >> ~/.zshrc && source ~/.zshrc

# 需要先获取 阿里云的access key： https://ram.console.aliyun.com/manage/ak
# 如果没有，则需要新建一对key 和secret

ALIBABA_CLOUD_ACCESS_KEY_ID = os.environ.get(
    "ALIBABA_CLOUD_ACCESS_KEY_ID", "请配置：ALIBABA_CLOUD_ACCESS_KEY_ID"
)  # 也可以直接把ACCESS_KEY_ID放在这里

ALIBABA_CLOUD_ACCESS_KEY_SECRET = os.environ.get(
    "ALIBABA_CLOUD_ACCESS_KEY_SECRET", "请配置：ALIBABA_CLOUD_ACCESS_KEY_SECRET"
)  # 也可以直接把ACCESS_KEY_SECRET放在这里

# 配置阿里云SDK：
config = open_api_models.Config(
    access_key_id=ALIBABA_CLOUD_ACCESS_KEY_ID,  # 请在阿里云上创建自己的ACCESS_KEY
    access_key_secret=ALIBABA_CLOUD_ACCESS_KEY_SECRET,  # 请在阿里云上创建自己的ACCESS_KEY_SECRET
)
# Endpoint 请参考 https://api.aliyun.com/product/devops
config.endpoint = f"devops.cn-hangzhou.aliyuncs.com"
client = devops20210625Client(config)

organization_id = "6189f099041d450d2c253abc"  # 这里固定值，为公司在云效上的唯一ID

project_id = "a41d2acc436b9755f11b79d3d4"  # 云效项目ID。这里每个组（供应链、SaaS/商城）不一样，可以通过云效果链接获取到，比如:
# https://devops.aliyun.com/projex/project/a41d2acc436b9755f11b79d3d4/task#viewIdentifier=6c6e9be7f1bafb355bff57fefa
# 以上链接中project/a41d2acc436b9755f11b79d3d4/task 即可拿到project_id；

# 先获取当前用户的名字：
res=client.get_user_info(request=devops_20210625_models.GetUserInfoRequest(organization_id=organization_id))
my_name=res.body.to_map()['result']['name']
logging.info(f"你的大名:{my_name}")

# 获取用户的ID：
res=client.list_organization_members(organization_id=organization_id, request=devops_20210625_models.ListOrganizationMembersRequest(organization_member_name=my_name))
members=res.body.to_map()
if len(members)<=0 or 'members' not in members:
    raise Exception(f"找不到用户:{my_name}")
else:
    members=members['members'][0]
    my_aliyun_pk=members['accountId']
    logging.info(f"当前用户:{members}")

# logging.info(ALIBABA_CLOUD_ACCESS_KEY_ID, ALIBABA_CLOUD_ACCESS_KEY_SECRET, my_aliyun_pk)

## 创建任务并且录入工时

- 任务仅可用作日常任务的创建；
- 项目任务仍然需要在对应的《需求》下创建并及时录入工时

In [ ]:
from datetime import datetime
import json

gmt_start = datetime.now().strftime("%Y-%m-%d")
worktime = "1"
time_type = {
    "研发": "develop",
    "测试": "test",
    "其他": "others",
    "联调": "85d41d4298fe223ae4313c2cd4",
    "冒烟": "e5060c97d239c8a957642deac8",
    "设计": "3a19b39714b0d0cd86ffc95687",
}


def create_task_for_me(task={}, gmt_start=gmt_start):
    subject = task["任务名"]
    worktime = task["工时"]
    task_type = "SaaS"
    if "归属业务" in task:
        logging.info(f"设置单独的Task Type:{task}")
        task_type = task["归属业务"]
    sprint_id = "fe2afd768b09b7e8fde816a21c"  # 其他事项
    if "会议" in subject or "周会" in subject:
        logging.info(f"这是一个会议：{subject}")
        sprint_id = "019dc37e2d25e42d6b5c6cd81e"  # 各种会议

    logging.info(f"===\n将为你创建任务：{subject}")

    field_value_list = []
    field_value_list.append(
        devops_20210625_models.CreateWorkitemV2RequestFieldValueList(
            field_identifier="fba0a046ffed2f9c5224db5493", value="SaaS"
        )
    )  # 业务模块
    field_value_list.append(
        devops_20210625_models.CreateWorkitemV2RequestFieldValueList(
            field_identifier="79", value=f"{gmt_start} 00:00:00"
        )
    )  # 计划开始时间
    field_value_list.append(
        devops_20210625_models.CreateWorkitemV2RequestFieldValueList(
            field_identifier="80", value=f"{gmt_start} 00:00:00"
        )
    )  # 计划完成时间
    # field_value_list.append(devops_20210625_models.CreateWorkitemV2RequestFieldValueList(field_identifier='0c906d95f677f4bd1c5d75e4a7',value="技术类需求")) # "技术类需求"
    field_value_list.append(
        devops_20210625_models.CreateWorkitemV2RequestFieldValueList(
            field_identifier="101586", value=worktime
        )
    )  # 预估工时
    field_value_list.append(
        devops_20210625_models.CreateWorkitemV2RequestFieldValueList(
            field_identifier="priority", value="e5d3d463361fe45fafc01a3b8a"
        )
    )  # 优先级

    # 创建一个新的空的任务
    req = devops_20210625_models.CreateWorkitemV2Request(
        assigned_to=my_name,
        category="Task",
        description=f"自动创建于:{datetime.now()}",
        field_value_list=field_value_list,
        space_identifier="a41d2acc436b9755f11b79d3d4",  # 项目空间
        sprint_identifier=sprint_id,  # 迭代ID
        subject=f"{subject}-{gmt_start}",
        # tags=["测试"],
        workitem_type_identifier="ba102e46bc6a8483d9b7f25c",
    )

    res = client.create_workitem_v2(request=req, organization_id=organization_id)
    new_item = res.body.to_map()
    pretty_output = json.dumps(new_item, indent=4)
    logging.info(f"创建新任务成功:\n{pretty_output}")

    work_item_id = new_item["workitemIdentifier"]

    # 更新任务的计划开始时间
    req = devops_20210625_models.UpdateWorkItemRequest(
        identifier=work_item_id,
        property_key="79",
        property_value=f"{gmt_start} 00:00:00",
        field_type="customField",
    )
    res = client.update_work_item(request=req, organization_id=organization_id)
    logging.info(f"更新任务的计划开始时间 成功:{res.body.to_map()['errorCode']}")

    # 更新任务的计划完成时间
    req = devops_20210625_models.UpdateWorkItemRequest(
        identifier=work_item_id,
        property_key="80",
        property_value=f"{gmt_start} 00:00:00",
        field_type="customField",
    )
    res = client.update_work_item(request=req, organization_id=organization_id)
    logging.info(f"更新任务的计划完成时间 成功:{res.body.to_map()['errorCode']}")

    # 更新任务的业务线
    req = devops_20210625_models.UpdateWorkItemRequest(
        identifier=work_item_id,
        property_key="fba0a046ffed2f9c5224db5493",
        property_value=task_type,
        field_type="customField",
    )
    res = client.update_work_item(request=req, organization_id=organization_id)
    logging.info(f"更新任务的业务线 成功:{res.body.to_map()['errorCode']}")

    # 更新任务的任务类型
    # req=devops_20210625_models.UpdateWorkItemRequest(identifier=work_item_id,property_key='0c906d95f677f4bd1c5d75e4a7',property_value='技术类需求',field_type='customField')
    # res=client.update_work_item(request=req,organization_id=organization_id)
    # logging.info(res.body.to_map())

    # 更新任务的优先级
    req = devops_20210625_models.UpdateWorkItemRequest(
        identifier=work_item_id,
        property_key="priority",
        property_value="e5d3d463361fe45fafc01a3b8a",
        field_type="customField",
    )
    res = client.update_work_item(request=req, organization_id=organization_id)
    logging.info(f"更新任务的优先级 成功:{res.body.to_map()['errorCode']}")

    # 新增任务的预计工时
    # req = devops_20210625_models.CreateWorkitemEstimateRequest(
    #     description=f"自动创建于:{datetime.now()}",
    #     record_user_identifier=my_aliyun_pk,
    #     spent_time=worktime,
    #     type=time_type["研发"],
    #     workitem_identifier=work_item_id,
    # )
    # res = client.create_workitem_estimate(request=req, organization_id=organization_id)
    # logging.info(res.body.to_map())

    # 新增任务的实际工时
    req = devops_20210625_models.CreateWorkitemRecordRequest(
        actual_time=worktime,
        description=f"自动创建于:{datetime.now()}",
        gmt_end=f"{gmt_start}",
        gmt_start=f"{gmt_start}",
        record_user_identifier=my_aliyun_pk,
        type=time_type["研发"],
        workitem_identifier=work_item_id,
    )
    res = client.create_workitem_record(request=req, organization_id=organization_id)
    logging.info(f"新增任务的实际工时 成功:{res.body.to_map()['errorCode']}")

    # 更新状态任务为已完成
    req = devops_20210625_models.UpdateWorkItemRequest(
        identifier=work_item_id,
        property_key="status",
        property_value="100014",
        field_type="status",
    )
    res = client.update_work_item(request=req, organization_id=organization_id)
    final_result = res.body.to_map()
    workitem=final_result['workitem']
    new_id=workitem['identifier']
    url=f"https://devops.aliyun.com/projex/project/a41d2acc436b9755f11b79d3d4/task#viewIdentifier=ab07d07c9654b95a147691f2&openWorkitemIdentifier={new_id}"
    logging.info(f"成功了: {workitem['serialNumber']}, {workitem['subject']}, {url}\n")

## 创建任务
* **任务名**：任务名称，本程序会自动在任务名后面加上日期
* **工时**：任务工时，支持小数，比如: 1.5, 2, 3
* **归属业务**：可选项目：'技术需求', 'SaaS', '全品类', '大客户', '鲜沐商城', 'CRM', '其他'

In [ ]:
task_type=['技术需求', 'SaaS', '全品类', '大客户', '鲜沐商城', 'CRM', '供应链计划（SCP）', '采购&供应商（PMS&SRM）', '履约&库存', 
           '仓储（WMS）', '仓网配送（WNC&TMS）', '供应链费用（BMS）', '供应链其他相关', '财务相关（应收&应付）', '其他', '鲜果POP']
tasks_to_create=[
    # {'任务名':"代下单性能优化-测试-下单数为0的SKU不校验报价单", '工时':'1.5','归属业务':'技术需求'},
    # {'任务名':"需求评审会议-营销价格展示迭代", '工时':'0.5','归属业务':'鲜沐商城'},
    {'任务名':"POP建品需求评审沟通会议", '工时':'1','归属业务':'鲜果POP'},
    {'任务名':"鲜沐商城限购漏洞修复", '工时':'3','归属业务':'鲜沐商城'},
    # {'任务名':"线上问题跟进-价格-上下架等等", '工时':'1','归属业务':'技术需求'},
    {'任务名':"鲜果POP匹配标果-增加类目的数据", '工时':'2','归属业务':'鲜果POP'},
    {'任务名':"SCP销量预测算法运用沟通会议-晓明", '工时':'1','归属业务':'其他'},
    # {'任务名':"商城/SaaS组周会", '工时':'1','归属业务':'技术需求'},
    ]
for task in tasks_to_create:
    create_task_for_me(task) # 创建当天的任务
    # create_task_for_me(task, gmt_start='2024-07-17') # 用来补任务（请尽力在下班前完成工时的录入）

In [ ]:
# import requests

# url = 'https://admin.summerfarm.net/cosfo-manage/product-agent-application/sku-agent-record/list'
# headers = {
#     'accept': 'application/json, text/plain, */*',
#     'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh-TW;q=0.7,zh;q=0.6',
#     'cache-control': 'no-cache',
#     'content-type': 'application/json',
#     'cookie': 'Hm_lvt_4c79036bfe07bfe7af679f50c1adaf0b=1716370731',
#     'origin': 'https://admin.summerfarm.net',
#     'pragma': 'no-cache',
#     'priority': 'u=1, i',
#     'referer': 'https://admin.summerfarm.net/summerfarm-fe/goods/agnet-audit/list',
#     'sec-ch-ua': '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
#     'sec-ch-ua-mobile': '?0',
#     'sec-ch-ua-platform': '"macOS"',
#     'sec-fetch-dest': 'empty',
#     'sec-fetch-mode': 'cors',
#     'sec-fetch-site': 'same-origin',
#     'token': 'admin__fc0a26ff-1221-4544-9d2b-8290607334d4',
#     'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
#     'xm-biz': 'xm-manage',
#     'xm-phone': '18618107293',
#     'xm-platform': 'web',
#     'xm-rqid': '1719986733653-61527871',
#     'xm-uid': '2047',
# }

# data = {
#     "pageIndex": 1,
#     "pageSize": 200,
#     "queryTenantName": "椿风",
#     "status": 1
# }

# response = requests.post(url, headers=headers, json=data).json()

# print(response)

In [ ]:
# import pandas as pd

# products_df=pd.DataFrame(response.get("data").get("list"))
# products_df.head(2)
# products_df.to_csv("./椿风鲜沐代仓所有货品.csv", index=False)